In [8]:
import os
import pandas as pd

# Girdi dosyalarının bulunduğu klasörü belirtin
input_folder = "Quotes"  # Klasör adını ayarlayın
output_csv = "philosopher_quotes.csv"  # Çıktı CSV dosyası

# Tüm .txt dosyalarını bul
text_files = [f for f in os.listdir(input_folder) if f.endswith(".txt")]

# Verileri tutmak için liste
data = []

# Her bir .txt dosyasını oku ve işleyip listeye ekle
for file in text_files:
    file_path = os.path.join(input_folder, file)

    # Dosyayı aç ve oku
    with open(file_path, "r", encoding="latin-1") as f:
        lines = f.readlines()

    # İlk satır filozofun adıdır
    philosopher_name = lines[0].strip()

    # Geri kalan satırlar sözlerdir
    quotes = [line.strip() for line in lines[1:] if line.strip()]

    # Her bir özlü sözü listeye ekle
    for quote in quotes:
        data.append({"Philosopher": philosopher_name, "Quote": quote})

# DataFrame'e dönüştür
df = pd.DataFrame(data)

# Veriyi CSV dosyasına yaz
df.to_csv(output_csv, index=False, encoding="utf-8")

print(f"Tüm dosyalar başarıyla {output_csv} dosyasına dönüştürüldü!")

Tüm dosyalar başarıyla philosopher_quotes.csv dosyasına dönüştürüldü!


In [9]:
print(df.head(10))

                                         Philosopher  \
0  In the small matters trust the mind, in the la...   
1  In the small matters trust the mind, in the la...   
2  In the small matters trust the mind, in the la...   
3  In the small matters trust the mind, in the la...   
4  In the small matters trust the mind, in the la...   
5  In the small matters trust the mind, in the la...   
6  In the small matters trust the mind, in the la...   
7  In the small matters trust the mind, in the la...   
8  In the small matters trust the mind, in the la...   
9  In the small matters trust the mind, in the la...   

                                               Quote  
0  Unexpressed emotions will never die. They are ...  
1  One day, in retrospect, the years of struggle ...  
2  We choose not randomly each other. We meet onl...  
3  Most people do not really want freedom, becaus...  
4  Out of your vulnerabilities will come your str...  
5  Words have a magical power. They can either br... 

In [10]:
import pandas as pd

# CSV dosyasını oku
input_csv = "philosopher_quotes.csv"
output_csv = "quotes_only.csv"

df = pd.read_csv(input_csv)

# 'Philosopher' sütununu kaldır
df = df.drop(columns=["Philosopher"], errors="ignore")

# Temizlenmiş CSV'yi kaydet
df.to_csv(output_csv, index=False, encoding="utf-8")

print(f"Filozof sütunu kaldırıldı. Yeni dosya: {output_csv}")

Filozof sütunu kaldırıldı. Yeni dosya: quotes_only.csv


In [11]:
import pandas as pd

# CSV dosyasını oku
df = pd.read_csv("quotes_only.csv")

# Quote sütunundaki boş satırları çıkar ve listeye dönüştür
quotes = df["Quote"].dropna().tolist()

print(f"Toplam {len(quotes)} adet özlü söz yüklendi.")

Toplam 2449 adet özlü söz yüklendi.


In [12]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer=GPT2Tokenizer.from_pretrained("distilgpt2")
model=GPT2LMHeadModel.from_pretrained("distilgpt2")

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader

class QuotesDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=50):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        tokens = self.tokenizer(
            text,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        return {
            "input_ids": tokens["input_ids"].squeeze(0),
            "attention_mask": tokens["attention_mask"].squeeze(0),
            "labels": tokens["input_ids"].squeeze(0),
        }

# Dataset ve DataLoader
dataset = QuotesDataset(quotes, tokenizer)
train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./distilgpt2-quotes",
    overwrite_output_dir=True,
    num_train_epochs=3,  # Eğitim sayısı
    per_device_train_batch_size=8,  # Batch boyutu
    save_steps=500,  # Modeli kaç adımda bir kaydedecek
    save_total_limit=2,  # Maksimum kaç checkpoint saklanacak
    logging_dir="./logs",  # Log dosyalarının kaydedileceği dizin
    logging_steps=100,  # Kaç adımda bir log yazılacak
    evaluation_strategy="no",  # Değerlendirme yapılmayacak
    learning_rate=5e-5,  # Öğrenme hızı
    weight_decay=0.01,  # Ağırlık çürümesi
    warmup_steps=100,  # Isınma adımı
    fp16=True,  # GPU kullanımı için
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

# Eğitimi başlat
trainer.train()

<ipython-input-15-317a15cdc669>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
100,3.417500
200,2.207200
300,2.127900
400,1.944000
500,2.011000
600,1.961100
700,1.868100
800,1.897100
900,1.855700


TrainOutput(global_step=921, training_loss=2.1377186562934734, metrics={'train_runtime': 211.5914, 'train_samples_per_second': 34.723, 'train_steps_per_second': 4.353, 'total_flos': 93737657548800.0, 'train_loss': 2.1377186562934734, 'epoch': 3.0})

In [16]:
# Eğitilmiş modeli kaydet
model.save_pretrained("./distilgpt2-quotes-final")
tokenizer.save_pretrained("./distilgpt2-quotes-final")

print("Eğitim tamamlandı ve model kaydedildi.")

Eğitim tamamlandı ve model kaydedildi.


In [25]:
from transformers import pipeline

# Eğitilmiş modeli tekrar yükle
generator = pipeline("text-generation", model="./distilgpt2-quotes-final", tokenizer=tokenizer)

# Yeni özlü sözler üret
outputs = generator("Life is", max_length=100, num_return_sequences=1)

# Üretilen metinleri yazdır
for i, output in enumerate(outputs):
    print(f"Özlü Söz {i+1}: {output['generated_text']}")

Device set to use cuda:0


Özlü Söz 1: Life is the capacity to have such a profound effect on our mind and to know without having our eyes or ears shut shut.


In [21]:
import shutil

# Modeli zip formatında sıkıştır
shutil.make_archive("/content/distilgpt2-quotes-final", 'zip', "/content", "distilgpt2-quotes-final")

print("Model zip dosyası başarıyla oluşturuldu.")

Model zip dosyası başarıyla oluşturuldu.


In [23]:
from google.colab import files

# Zip dosyasını indir
files.download("/content/distilgpt2-quotes-final.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>